In [1]:
#!pip install tensorflow_addons

# Imports

In [ ]:
# !pip install -q tensorflow tensorflow-addons scikit-learn

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from scipy.io import arff
import pandas as pd
from tensorflow.keras.utils import to_categorical


from scipy.io import arff





In [3]:
import pickle

from google.colab import files
#uploaded = files.upload()
#Todo 1.0
#!wget -O /content/ECG5000_train.pickle /content/sample_data/ECG5000_train (5).pickle

with open("/content/sample_data/ECG5000_train (5).pickle", 'rb') as f:
  ecg_train = pickle.load(f)
#!wget -O /content/ECG5000_train.pickle /content/sample_data/ECG5000_validation (3).pickle

with open("/content/sample_data/ECG5000_validation (3).pickle", 'rb') as f:
  ecg_validation = pickle.load(f)

# Checking data

# Feature label split and standardization

In [4]:
swapTrainvld = True

# Data imbalance.

Here we try to increase minority class samples with few of the techniques.
Noise plus minus

1.   Plus Minus noise: Adding or substracting small noise from the row to get more sample. Idea is that it will help capture underlying patterns of minority class.  
2.   Smothing odd even: Similar to animation industry where animators draw around 12 frames for the animaiton of video 24frames per second. As drawing all frames would make animation shake a lot and unstable. Similar concept was used here to smooth out samples and make more samples from it self.
This works similar to
<br>Sample = [1,2,4,2,5,1]
<br>oddsample = [1,1,4,4,5, 5]
<br>evensample = [2,2,2,2,1,1]<br>
This way are are able to get 3 samples from one original one. The example many not be as good but it is similar in working. It can also cause side effects like missing out peaks and valleys in sample. But it has worked little better then making model heavily baised towards minority samples.
For example, if majority class had 100 samples, we would need around 700 duplicates of minority samples to get good f1 compared to raw.



  

In [5]:
import numpy as np
def augment_minority_with_noise_and_smoothing(x_train, y_train, classes_to_augment=[2, 3, 4], noise_std=0.05, target_ratio=1.3,
                           dual_noise=True, use_smoothing=True, shuffle=True, random_state=42):

    y_train = y_train.squeeze().astype(np.int32)
    class_counts = np.bincount(y_train)
    max_class_count = int(np.max(class_counts) * target_ratio)

    x_aug = [x_train]
    y_aug = [y_train]

    rng = np.random.default_rng(random_state)

    for cls in classes_to_augment:
        if cls >= len(class_counts):
            continue

        count = class_counts[cls]
        if count >= max_class_count:
            continue

        n_needed = max_class_count - count
        x_cls = x_train[y_train == cls]

        new_samples = []
        new_labels = []

        while len(new_samples) < n_needed:
            for sample in x_cls:
                # plus minus noise
                noise = rng.normal(0, noise_std, size=sample.shape)
                new_samples.append(sample + noise)
                new_labels.append(cls)
                if dual_noise:
                    new_samples.append(sample - noise)
                    new_labels.append(cls)

                # odd even smoothing
                if use_smoothing:
                    even_smoothed = sample.copy()
                    odd_smoothed = sample.copy()

                    # Repeating even values into odd positions
                    even_smoothed[1::2] = even_smoothed[::2][:len(even_smoothed[1::2])]
                    new_samples.append(even_smoothed)
                    new_labels.append(cls)

                    # Repeating odd values into even positions
                    if len(sample) > 1:
                        odd_smoothed[::2] = odd_smoothed[1::2][:len(odd_smoothed[::2])]
                        new_samples.append(odd_smoothed)
                        new_labels.append(cls)

                if len(new_samples) >= n_needed:
                    break

        x_aug.append(np.array(new_samples[:n_needed]))
        y_aug.append(np.array(new_labels[:n_needed]))

    x_final = np.concatenate(x_aug, axis=0)
    y_final = np.concatenate(y_aug, axis=0)

    if shuffle:
        indices = np.arange(len(y_final))
        rng.shuffle(indices)
        x_final = x_final[indices]
        y_final = y_final[indices]

    return x_final, y_final


This is just used for printing data types, shapes, and unique values for debugging purposes.

In [6]:
def printInfo(X_trn, y_trn, mesage = "Trn"):
  print("\n")

  try:
    if y_trn.shape[1] > 1:
      counts = np.sum(y_trn, axis=0)
      uniques = np.arange(y_trn.shape[1])
    else:
      print("Y is not one hot encoded here")
      uniques, counts = np.unique(y_trn.flatten(), return_counts = True)
  except:
    print("Y is not one hot encoded here")
    uniques, counts = np.unique(y_trn.flatten(), return_counts = True)
  print(mesage)
  print("Types: ")
  print(f"\tX_trn type = {type(X_trn)}, y_trn type = {type(y_trn)}")
  print("Shapes: ")
  print(f"\tX_trn.shape = {X_trn.shape}, y_trn.shape = {y_trn.shape}")
  print("Y unique plus counts")
  for i,j in zip(uniques, counts):
    print(f"\tClass = {i}, counts = {j}")



# Data agumentation

Method for duplicating minority samples

In [7]:
def augment_class(X, y, target_class, augment_count):
    class_indices = np.where(y == target_class)[0]
    chosen = np.random.choice(class_indices, augment_count, replace=True)
    return X[chosen], y[chosen]

# Data prep
This method applies all data cleaning process in one go. All the values have defaults so it can be called even without parameters.

Important parameters are
<br>agument_class_simple: which simply duplicates minority class.
<br>transform_columnWise: which transaform each feature independently.
<br>augment_with_odd_even: applies our very first class. Class to augment are hardcoded method

In [8]:
scaler = StandardScaler()
def data_prep(
    ecg_train: np.ndarray = ecg_train,
    ecg_validation: np.ndarray = ecg_validation,
    swap_train_vld: bool = True,
    augment_classes_simple: dict = {2: 290, 3: 200, 4: 500},
    transform_columnWise: bool = True,
    augment_with_odd_even: bool = False,
    verbose: bool = True
    ):


  #swap set
  #Checking if train and validation was swapped by mistake
  if ecg_train.shape[0] > ecg_validation.shape[0]:
      temp = ecg_train
      ecg_train = ecg_validation
      ecg_validation = temp
  #Swap vld train
  if swap_train_vld:
    swapTrainvld = True
    temp = ecg_train
    ecg_train = ecg_validation
    ecg_validation = temp

  #feature label split
  ecg_train_data = ecg_train[ : ,1:]
  ecg_train_labels = ecg_train[:,0:1]
  ecg_validation_data = ecg_validation[ : ,1:]
  ecg_validation_labels = ecg_validation[:,0:1]

  #reassign
  X_trn = ecg_train_data
  y_trn = ecg_train_labels
  X_vld = ecg_validation[:, 1:]
  y_vld = ecg_validation_labels
  if verbose: printInfo(X_trn, y_trn, "Initial train")
  if verbose: printInfo(X_trn, y_trn, "Initial validation")




  #Augment with odd_even
  if augment_with_odd_even:
    if verbose: printInfo(X_trn, y_trn, "Before augment_with_odd_even")
    y_trn = y_trn.squeeze()
    y_trn = y_trn.astype(np.int32)
    X_trn, y_trn = augment_minority_with_noise_and_smoothing(X_trn, y_trn)
    if verbose: printInfo(X_trn, y_trn, "After augment_with_odd_even")

  #Augment simple
  if augment_classes_simple:
    if verbose: printInfo(X_trn, y_trn, "Before augment_classes_simple")
    X_aug, y_aug = [], []
    aug_info = {2: 290, 3: 200, 4: 500}

    for cls, count in aug_info.items():
        x_new, y_new = augment_class(X_trn, y_trn, cls, count)
        X_aug.append(x_new)
        y_aug.append(y_new)

    X_trn = np.concatenate([X_trn] + X_aug)
    y_trn = np.concatenate([y_trn] + y_aug)
    if verbose: printInfo(X_trn, y_trn, "After augment_classes_simple")

    #Standardization

    if transform_columnWise:
      X_trn = scaler.fit_transform(X_trn)
      X_vld = scaler.transform(X_vld)
    else:
      X_trn  = X_trn.T
      X_vld = X_vld.T
      X_trn = scaler.fit_transform(X_trn)
      X_vld = scaler.fit_transform(X_vld)
      X_trn  = X_trn.T
      X_vld = X_vld.T

    # Reshape for TCN [batch, time_steps, features]
    X_trn = X_trn.reshape(X_trn.shape[0], X_trn.shape[1], 1)
    X_vld = X_vld.reshape(X_vld.shape[0], X_vld.shape[1], 1)

    if verbose: printInfo(X_trn, y_trn, "Before One hot train")
    if verbose: printInfo(X_vld, y_vld, "Before One hot vald")
    num_classes = 5
    y_trn_oh = to_categorical(y_trn, num_classes)
    y_vld_oh = to_categorical(y_vld, num_classes)
    if verbose: printInfo(X_trn, y_trn_oh, "After One hot train")
    if verbose: printInfo(X_vld, y_vld_oh, "After One hot vald")

    return X_trn, y_trn_oh, X_vld, y_vld_oh, y_trn, y_vld





# returns:
#X_trn, y_trn_oh, X_vld, y_vld_oh, y_trn, y_vld
#X_trn, y_trn_oh, X_vld, y_vld_oh, _, _ = data_prep()

In [9]:
X_trn, y_trn_oh, X_vld, y_vld_oh, _, _ = data_prep(
    ecg_train = ecg_train,
    ecg_validation = ecg_validation,
    swap_train_vld = swapTrainvld, #set this manaully to false to see paper's score
    augment_classes_simple = {3: 100},#{0: 250, 1: 250, 2: 260, 3: 400, 4: 1000}, # increase minority class
    transform_columnWise = True,
    augment_with_odd_even = True, #increasing minority classes
    verbose = True
    )



Y is not one hot encoded here
Initial train
Types: 
	X_trn type = <class 'numpy.ndarray'>, y_trn type = <class 'numpy.ndarray'>
Shapes: 
	X_trn.shape = (1500, 140), y_trn.shape = (1500, 1)
Y unique plus counts
	Class = 0.0, counts = 781
	Class = 1.0, counts = 590
	Class = 2.0, counts = 43
	Class = 3.0, counts = 75
	Class = 4.0, counts = 11


Y is not one hot encoded here
Initial validation
Types: 
	X_trn type = <class 'numpy.ndarray'>, y_trn type = <class 'numpy.ndarray'>
Shapes: 
	X_trn.shape = (1500, 140), y_trn.shape = (1500, 1)
Y unique plus counts
	Class = 0.0, counts = 781
	Class = 1.0, counts = 590
	Class = 2.0, counts = 43
	Class = 3.0, counts = 75
	Class = 4.0, counts = 11


Y is not one hot encoded here
Before augment_with_odd_even
Types: 
	X_trn type = <class 'numpy.ndarray'>, y_trn type = <class 'numpy.ndarray'>
Shapes: 
	X_trn.shape = (1500, 140), y_trn.shape = (1500, 1)
Y unique plus counts
	Class = 0.0, counts = 781
	Class = 1.0, counts = 590
	Class = 2.0, counts = 43


Using relu instead of sigmoid, increasing layers to 32 from 16. Changing dropout to 0.3 to prevent overfitting.

In [10]:
def build_tcn_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    filter_sizes = [2, 4, 6, 8]

    for i, fs in enumerate(filter_sizes):
        dilation_rate = 2 ** i
        x_prev = x
        x = layers.Conv1D(32, fs, padding='causal', dilation_rate=dilation_rate, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.SpatialDropout1D(0.3)(x)
        x = layers.Add()([x, x_prev]) if x.shape[-1] == x_prev.shape[-1] else x

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model


In [11]:
from sklearn.metrics import f1_score
import numpy as np
import tensorflow as tf
# Adding weights to minority class.
class_weight_dict = {
    0: 50.0,
    1: 40.0,
    2: 30.5,
    3: 40.0,
    4: 60.0
}

# Instead of relying on accuracy, we directly calculate validation f1 score per epoch and print it.
# If model reachs our desired socre it stops training.
class F1EarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, val_data, target_f1=0.90):
        super().__init__()
        self.X_val, self.y_val = val_data
        self.target_f1 = target_f1
        self.best_f1 = 0


    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_val, verbose=0)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = np.argmax(self.y_val, axis=1)

        current_f1 = f1_score(y_true, y_pred_classes, average='macro')
        logs['val_f1_score'] = current_f1

        print(f" — val_f1_score: {current_f1:.4f}", end='')


        if current_f1 >= self.target_f1:
            print(f"\nEarly stopping - F1 reached {current_f1:.4f} (target: {self.target_f1})")
            self.model.stop_training = True

        if current_f1 > self.best_f1:
            self.best_f1 = current_f1
            print(f"\tEpoch = {epoch}")
            print(" (new best)", end='')
        print()
# Defining early stopping since we may not get same performance even after finding
# epoch number because shuffle = True when running the model.
if swapTrainvld:
  setTargetF1 = 0.85
else:
  setTargetF1 = 0.65
f1_callback = F1EarlyStopping(
    val_data=(X_vld, y_vld_oh),
    target_f1=setTargetF1
)
print(f"Target f1 for early stopping = {setTargetF1}")

Target f1 for early stopping = 0.85


In [ ]:
model = build_tcn_model(X_trn.shape[1:], 5)#num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_trn, y_trn_oh,
    epochs=400,
    batch_size=24,
    validation_data=(X_vld, y_vld_oh),
    callbacks=[f1_callback],
    verbose=0, # Because of hardware limitions, verbose has been turned off to prevent PC from hanging.
    class_weight = class_weight_dict
)


 — val_f1_score: 0.2108	Epoch = 0
 (new best)
 — val_f1_score: 0.4999	Epoch = 1
 (new best)
 — val_f1_score: 0.4930
 — val_f1_score: 0.6190	Epoch = 3
 (new best)
 — val_f1_score: 0.5950
 — val_f1_score: 0.6810	Epoch = 5
 (new best)
 — val_f1_score: 0.6954	Epoch = 6
 (new best)
 — val_f1_score: 0.7010	Epoch = 7
 (new best)
 — val_f1_score: 0.7556	Epoch = 8
 (new best)
 — val_f1_score: 0.7783	Epoch = 9
 (new best)


In [ ]:
y_pred_probs = model.predict(X_vld)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_vld_oh, axis=1)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("F1 Score (macro):", f1_score(y_true, y_pred, average='macro'))
print("\n", classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))



Based on results and based on KNN classifier from earlier project where we had class imbalance we can trust models instinct, ie first preference for class 0 and 1.
In other words, if the model says that the given sample is class 0 and 1 we can trust blindly. If model says that given sample is 2-3 then we cannot blindly trust model as it didn't see enough data for class 2-3 and almost nothing for 4. But model did see some samples of minority classes so it also means it learned something.

In [ ]:
X_trn, _, X_vld, _, _, y_vld =  data_prep(
    ecg_train = ecg_train,
    ecg_validation = ecg_validation,
    swap_train_vld = swapTrainvld,
    augment_classes_simple = {3: 100}, # This augmentation wasn't working that great so we only doing it for class one.
    transform_columnWise = True,
    augment_with_odd_even = True, #increasing minority classes with repeatition
    verbose = False
    )


We try to pick random samples from predicted samples. Idea is that since model is good with class 1 and 2 we leave it alone. Since class 2 and 3 are minority and 4 is extreme minority, we randomly pick data samples class 2 and 3 since they are in middle and assign them manually to other classes. Since we need some pattern, we convert probabilities to ranking.
So every sample will now have ranking of class preferences they would like to be in.
<br>For example
<br>Class idx (0,1,2,3,4)
<br>Sample1 = (1,2,0,4,3)<br>
Ranking is reversed so lowest ranking = highest probability and vica-versa.<br>
This sample will be predicted as class 2.

Idea is that, if model is good in identifying samples and if model says given sample is 0,1 we can trust models 1st instinct which is mostly right.
If it says it's 2 or 3 we can't trust it's 1st instinct/preference. But we also don't know if we can trust it's 2nd, 3rd or 4th instinct/preference when it predicts 2 or 3. Also we won't know which know which class to override from

So we use nested for loops to find combination of class to overRide samples with, and sample's preference for that class. This might be similar to threshold tuning on validation set, but instead we are doing preference based post processing of results.
Since softmax will try to make incorrect class probabilities lower, we convert them into rank and find prefences of model.

Theory is that, since we are being told, before speaking in important situation, we should not say 1st,2nd of 3rd thing that comes to our mind instead 4th one. Because most of the times anything after 1st thinking is relatively more rational thinking then 1st one.
If we apply same logic to model, then we want to find preference/thinking of that model and class to apply those perferences to.


In [ ]:
def overRidePredictLoop (X_vld, y_vld, class_overRide = 3, sample_preference = 3, return_value = False):
  # Regular predictions
  y_pred_probs = model.predict(X_vld)
  y_pred = np.argmax(y_pred_probs, axis=1)
  y_pred_original = y_pred.copy()

  # converting probabilities into ranking
  rankings = np.argsort(-y_pred_probs, axis=1)

  # Appling target override. If any sample is predicted as 2 or 3, we loop through their
  # preferences for each class and class to overRide that sample. Here we have possible of
  # 16 combinations 4 * 4 since we are not considering override on class 0. If the model
  # is performaning perfectly, this method will report 0.00 increase in performance, meaning there is no
  # possible combination of class_overRide amd sample_preference which can further increase it's score.
  # However these are still overRidden scores for f1 and not something that model has learned.

  eligible_indices = [
      i for i in range(len(y_pred))
      if y_pred[i] in [2, 3] and rankings[i][class_overRide] == sample_preference
  ]
  eligible_indices = np.array(eligible_indices, dtype=int)

  np.random.seed(42)
  override_indices = np.random.choice(eligible_indices, size=min(4000, len(eligible_indices)), replace=False)

  y_pred[eligible_indices] = class_overRide

  overRide = f1_score(y_vld, y_pred, average='macro')
  original = f1_score(y_vld, y_pred_original, average='macro')
  result = overRide - original
  if return_value:
    return result
  else:
    # Evaluate new predictions
    print("After Targeted Override ")
    print(confusion_matrix(y_vld, y_pred))
    print(classification_report(y_vld, y_pred, digits=4))
    print("\nOriginal Macro F1:", f1_score(y_vld, y_pred_original, average='macro'))
    print("\n\n\n")
    print("🔻NOT MODELS PERFORMANCE REAL PERFORMANCE ")
    print("Override Macro F1 Score:", f1_score(y_vld, y_pred, average='macro'))
    print("🔺NOT MODELS PERFORMANCE REAL PERFORMANCE ")
    if result < 0:
      print(f"Performance drop = {result}")
    else:
      print(f"Performance increase = {result}")





#print(overRidePredictLoop(X_vld, y_vld, class_overRide=3, sample_preference = 4))

In [ ]:

def findBestOverRide(X_vld, y_vld):
  class_overRide = 0
  sample_preference= 0
  best_f1 = -100
  for i in range(1,5):
    for j in range(1,5):
      current_f1 = overRidePredictLoop(X_vld, y_vld, class_overRide = i, sample_preference = j, return_value=True)
      if current_f1 > best_f1:
        print(f"curent difference = {current_f1}")
        best_f1 = current_f1
        class_overRide = i
        sample_preference = j
        print(f"New best f1, increase of  = {best_f1}, for class_override = {i}, and sample_preference = {j}")
  return class_overRide, sample_preference


class_overRide, sample_preference = findBestOverRide(X_vld, y_vld)


In [ ]:
#final over ride
overRidePredictLoop(X_vld, y_vld, class_overRide = class_overRide, sample_preference = sample_preference)

This was implementation of our Method A from  https://pmc.ncbi.nlm.nih.gov/articles/PMC9920651/ paper .
Their original score was
f1 = 84%, accuracy = 96% with data agumentation on minority classes like 2,3,4.

Things used from papers were concecpts of TCN model, data augmentation, class weights.

My condtributions: My contribution to this was




**DataAugmentation:** Instead of using standard data augmentation, we used noise based plus minus data augmentation methods along with odd even smoothing which not only creates more samples for minority class, but those extra samples are also smoothed out, which helps might have helped in capturing underlying patters of minority classes and contributed in increasing performance.

There was 1% F1 increase on validation's original socre when we augmented class 3 aswell along with 2 and 4. Along with that our model converged relatively early then before.

**overRidePredictions()** which increase performance artificaly if it can for the given validation set by finding the best combination of class to overRide on and preference to consider. This resulted in 0.77% increase in artifically bosted performance.

Apart from that, training on raw data without any augmentations gets around 4.00% f1 score boost from original data set. However for that, we must train it on raw data without any class weights or data augmentation.
One limitation of this is that while it doesn't decrease f1 because it excludes class 0 and 1, and only considers 2 and 3, it also means that currently if model predicts minority samples are class 0 or 1, this method can't work with that.

Another limitations of this artifical boosting method is that while working further on this, or in real life cases, we must ensure that test data set will follow roughly same class distribution as it's seen in training data sets. One way to achieve these would be to gather samples from differnt places and have them predicted at once in batch. That might be the only way to get reliable results from these artifical boost.

In real life implementation for example in hospitals, they must wait for set period of time until there are atleast enough samples to test from when we would be sure that those samples might follow natural pattern that was in training data set. Wether these artifical boosts are okay in machine learning world or not might be in grey area. But there is one thing that, it might open up new way of working with model's flaw. Instead of perfecting model, we work with their flaws to get best performance for each class.









In [ ]:
"""

from sklearn.utils import class_weight
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

class_weights_original_0 = {0: 0.1459, 1: 0.1041}
class_weights_original_1 = {0: 0.884, 1: 0.157}
class_weights_original_2 = {0: 1.97,   1: 20.60}
class_weights_original_3 = {0: 10.48,  1: 25.12}
# for binary
y_trn_0 = np.argmax(y_trn_0, axis = 1).reshape(-1,1)
y_trn_1 = np.argmax(y_trn_1, axis = 1).reshape(-1,1)
y_trn_2 = np.argmax(y_trn_2, axis = 1).reshape(-1,1)
y_trn_3 = np.argmax(y_trn_3, axis = 1).reshape(-1,1)


def build_tcn_model(input_shape, num_classes=2):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    filter_sizes = [2, 4, 6, 8]

    for i, fs in enumerate(filter_sizes):
        dilation_rate = 2 ** i
        x_prev = x
        x = layers.Conv1D(16, fs, padding='causal', dilation_rate=dilation_rate, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.SpatialDropout1D(0.1)(x)
        x = layers.Add()([x, x_prev]) if x.shape[-1] == x_prev.shape[-1] else x

    x = layers.GlobalAveragePooling1D()(x)
    #outputs = layers.Dense(num_classes, activation='softmax')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = models.Model(inputs, outputs)
    return model

def get_class_weights(y):
    y_integers = np.argmax(y, axis=1)
    weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_integers),
        y=y_integers
    )
    return dict(enumerate(weights))

print("Model 0")
model0 = build_tcn_model(X_trn_0.shape[1:], num_classes=2)
model0.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
class_weights_0 = get_class_weights(y_trn_0)
history0 = model0.fit(
    X_trn_0, y_trn_0,
    epochs=19,
    batch_size=20,
    validation_split=0.1,
    verbose=1,
    class_weight= class_weights_original_0
)

print("Model 1")
model1 = build_tcn_model(X_trn_1.shape[1:], num_classes=2)
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
class_weights_1 = get_class_weights(y_trn_1)
history1 = model1.fit(
    X_trn_1, y_trn_1,
    epochs=12,
    batch_size=20,
    validation_split=0.1,
    verbose=1,
    class_weight=class_weights_original_1
)

print("Model 2")
model2 = build_tcn_model(X_trn_2.shape[1:], num_classes=2)
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
class_weights_2 = get_class_weights(y_trn_2)
history2 = model2.fit(
    X_trn_2, y_trn_2,
    epochs=50,
    batch_size=10,
    validation_split=0.1,
    verbose=1,
    class_weight=class_weights_original_2
)

print("Model 3")
model3 = build_tcn_model(X_trn_3.shape[1:], num_classes=2)
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
class_weights_3 = get_class_weights(y_trn_3)
history3 = model3.fit(
    X_trn_3, y_trn_3,
    epochs=50,
    batch_size=5,
    validation_split=0.1,
    verbose=1,
    class_weight=class_weights_original_3
)



def Splitter(X_trn, y_trn):
    features = X_trn.shape[1]

    X_0, y_0 = [], []
    X_1, y_1 = [], []
    X_3, y_3 = [], []
    X_2, y_2 = [], []

    for i in range(len(X_trn)):
        x = X_trn[i]
        y = y_trn[i]

        # Stage 0: 0 vs 1,2,3,4
        X_0.append(x)
        y_0.append(0 if y == 0 else 1)

        if y != 0:
            # Stage 1: 1 vs 2,3,4
            X_1.append(x)
            y_1.append(0 if y == 1 else 1)

            if y != 1:
                # Stage 3: 3 vs 2,4
                if y in [2, 3, 4]:
                    X_3.append(x)
                    y_3.append(0 if y == 3 else 1)

                # Stage 2: 2 vs 4 (only if passed stage 3 as not class 3)
                if y in [2, 4] and y != 3:
                    X_2.append(x)
                    y_2.append(0 if y == 2 else 1)

    # Convert to numpy arrays
    return (
        np.array(X_0), np.array(y_0),
        np.array(X_1), np.array(y_1),
        np.array(X_3), np.array(y_3),
        np.array(X_2), np.array(y_2)
    )



def predictmerger()
"""